# Table of Contents
 <p><div class="lev1 toc-item"><a href="#RESTCONF" data-toc-modified-id="RESTCONF-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>RESTCONF</a></div><div class="lev1 toc-item"><a href="#Generic-Information-about-RESTCONF" data-toc-modified-id="Generic-Information-about-RESTCONF-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Generic Information about RESTCONF</a></div><div class="lev2 toc-item"><a href="#URI-construction" data-toc-modified-id="URI-construction-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>URI construction</a></div><div class="lev2 toc-item"><a href="#RESTCONF-and-NETCONF" data-toc-modified-id="RESTCONF-and-NETCONF-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>RESTCONF and NETCONF</a></div><div class="lev2 toc-item"><a href="#API-Catalog?" data-toc-modified-id="API-Catalog?-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>API Catalog?</a></div><div class="lev1 toc-item"><a href="#Postman" data-toc-modified-id="Postman-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Postman</a></div><div class="lev1 toc-item"><a href="#Code-Setup" data-toc-modified-id="Code-Setup-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Code Setup</a></div><div class="lev1 toc-item"><a href="#Device-Discovery" data-toc-modified-id="Device-Discovery-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Device Discovery</a></div><div class="lev1 toc-item"><a href="#Get-Capabilities" data-toc-modified-id="Get-Capabilities-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Get Capabilities</a></div><div class="lev1 toc-item"><a href="#Get-the-Interface-List-of-the-Device" data-toc-modified-id="Get-the-Interface-List-of-the-Device-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Get the Interface List of the Device</a></div><div class="lev1 toc-item"><a href="#Create-a-VLAN-Interface" data-toc-modified-id="Create-a-VLAN-Interface-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Create a VLAN Interface</a></div>

# RESTCONF
RESTCONF uses the same YANG models underneath as NETCONF does but uses a different transport and encapsulation. 

- Where NETCONF uses SSH, RESTCONF uses HTTP(S). 

- RESTCONF is, by its very nature, stateless. So, there is no concept of a *connection* at the RESTCONF level as opposed to NETCONF. 

- It also is not a finalized standard but still in a draft. See [here on the IETF site](https://tools.ietf.org/html/draft-ietf-netconf-restconf-18) for the current revision as of November 2016.

- It can use JSON as the data encapsulation method which is often preferred by programmers. 

In this notebook we're looking at a couple of examples how to programatically interact with RESTCONF enabled devices using our CSR1000V devices running IOS-XE.

# Generic Information about RESTCONF

## URI construction 

![URI Structure](../images/uri-structure.png)

(where M=mandatory, O=Optional and I=ignored). The API entry is derived from a well-known entry point on the device. In our specific case with the CSR1000V it is `[device-ip:80]` and then `/restconf/api`. 


![URI Structure](../images/url-construction.png)


<p class='my-notify-info'>For the sake of simplicity we're not using HTTPS with the devices but unencrypted HTTP. This is, of course, not recommended for production environments.</p>

## RESTCONF and NETCONF
The table below shows important HTTP verbs which are used with RESTCONF and how they compare to related commands used in NETCONF.

RESTCONF | Compared to NETCONF
---- | ----
GET | `<get-config>, <get>`
POST | `<edit-config> (operation="create")`
PUT | `<edit-config> (operation="create/replace")`
PATCH | `<edit-config> (operation="merge")`
DELETE | `<edit-config> (operation="delete")`

## API Catalog?
If you've ever dealt with other REST based APIs then you're probably wondering where the API catalog is (e.g. the 'directory' with all available API calls and their documentation).

In fact, **there is no API catalog in RESTCONF**. 

Essentially, RESTCONF defines how a YANG model is mapped to a RESTful interface. Specifically

- how to construct URIs to access the model / data,
- how to modify the data by using REST verbs,
    
    (`GET` / `PUT` / `PATCH` / ...),<p/>

- RESTCONF is about the process, a formalized way to talk to a device,
- RESTCONF knows a few ‘verbs’,
- RESTCONF uses data models to describe device capabilities and functions,
- RESTCONF provides well known entry points to ‘discover’ those.

Think ‘SNMP and MIBs’.

![URI Structure](../images/snmp-comparison.png)


# Postman
The Chrome extension [Postman](https://getpostman.com) is often referred to as the tool of choice when it comes to RESTCONF. And of course you can use Postman for the examples presented in this Notebook too. However, since this workbench is about prorgamming we are using the Python Requests library for our examples.

If you want to follow along with Postman simply make sure to copy paste the right items from the code to Postman. These are:

1. **Method**. Typically 'GET', 'POST' or 'PATCH'. Select from the drop down
2. **URL**. Obviously goes into the URL field
3. **Authentication data**. Username and password. Make sure to click the update button so that the Authorization header gets updated
4. **Headers**. Typicyall, 'Authorization', 'Content-Type' (what we send) and 'Accept' (what we expect as a result) are set
5. **Body**. For POST or PATCH methods, otherwise grayed-out. Should be set to 'raw' and typically contains either XML or JSON encoded data.
6. **Params**. Are added to the URL with a question mark and indicate some additional parameters to the call.

![Postman](../images/postman.png)

You'll find these values in the code snippets below so that you can copy / paste them directly from the code into Postman if you want to use Postman as an alternative to Python code or for some quick tests.


# Code Setup
This imports the Requests library and provides some common functions we will use all over the place. It also sets up a few global vars we will use through the remainder of the notebook

In [ ]:
import requests
import json

HOST = '172.20.20.10'
USER = PASS = 'vagrant'
PORT = 80
PROT = 'http'

API_JSON = 'application/vnd.yang.api+json'
DATA_JSON = 'application/vnd.yang.data+json'

def url(method=''):
    '''return the proper URL given the global vars and the
    method parameter.
    '''
    return "{prot}://{host}:{port}/restconf/api/{method}".format(prot=PROT, host=HOST, method=method, port=PORT)

def pp(data):
    '''pretty print the Python object
    '''
    if isinstance(data, requests.models.Response):
        if not data.ok:
            print('{}: {}'.format(data.status_code, data.text))
            return None
        obj = data.json()
    else:
        obj = data
    print(json.dumps(obj, indent=2))
    return obj

# Setup a global session with proper username and password
session = requests.Session()
session.auth = (USER, PASS)
# we also set the default dictionary with our 'Accept' header
session.headers = {'accept': DATA_JSON, 'content-type': DATA_JSON}

# are we using HTTPS?
if 'https' in PROT:
    # dont' verify certificates
    session.verify = False
    # disable warnings in case we use HTTPS
    requests.packages.urllib3.disable_warnings()

The URL below is the one that can be used with Postman as the 'Base URL':

In [ ]:
url('')

# Device Discovery
Device discovery is the top-level RESTCONF entry point. It assumes that we already know the API entry point which is, in the IOS-XE case, `/restconf/api` on port 80 or 443 (when using HTTPS).

Try to add `verbose` to the call by adding Query Parameters to the call. Query parameters use the `params=` parameter to the requests call and should be either a list or a dictionary.

Try:

    params=dict(verbose="1")
    
as the next parameter after `headers=...` in the call below for verbose output.
    
<p class="my-notify-info">The requests library can currently not pass any URL parameters without a value. The RESTCONF agent only needs '?verbose', we're passing '?verbose=1' but that doesn't really hurt in our case. See [this discussion](https://github.com/kennethreitz/requests/issues/2651) for more details.</p>

Note that we need to pass a different 'accept' header to this call.

In [ ]:
o = pp(session.get(url(), headers=dict(accept=API_JSON)))

# Get Capabilities

In [ ]:
o = pp(session.get(url('operational/netconf-state/capabilities')))

# Get the Interface List of the Device
From the previous exercise we have learned the API entry point (`/api`) and the prefix for operational data (`operational`) and configuration data (`config` and `running`). We also know that the device supports the `ietf-interfaces.yang` model.

<p class='my-notify-info'>Both configuration entry points, config and running, modify the configuration of the device. The difference between them is whether changes are persistent (config), or transient (running). The exact behavior / definition is part of quite some discussion in the IETF and subject to change.
Let’s get the interface list from the device.</p>

Select and run the ‘Read ietf-interfaces list of interface names’ call. The device will return the list of interfaces it knows about.

Again, try with `verbose` and / or `deep` as parameters to the call. (`pp(session.get(url('running/interfaces'), params=dict(deep=True)))`).

In [ ]:
o = pp(session.get(url('running/interfaces')))

Let's modify the URL to only retrieve the verbose information for a single interface. Refer to the `pyang` output from the NETCONF lab if you want to have a reference what the `ietf-interfaces.yang` model looks like.

In [ ]:
o = pp(session.get(url('running/if:interfaces/if:interface/Loopback99'), params=dict(deep=True)))

Let's compare this to the output we generated in the NETCONF section. As we can see, the path from the Javascript enabled HTML output relates direct to the path we put into the RESTCONF request's path element. The only thing that might be different is whether the namespace is explicitely provided or not.

As when looking at the NETCONF examples, providing the namespace can be useful to avoid ambiguities, e.g. when the same name is used within different models.

In [ ]:
%%HTML
<iframe width="100%" height="400px" src='/notebooks/tmp/ietf-interfaces.html'</iframe>

# Create a VLAN Interface

Again, we're using `pyang` to look at a YANG model, this time the *Native Element Driver* (NED) which is a device-specific YANG model for IOS-XE. We are specifically interested in the VLAN interface definition and the tree rendered below shows only that portion.

The following command has been used to create this tree:

```
pyang --jstree --jstree-path='/native/interface/GigabitEthernet'  yang/vendor/cisco/xe/ned.yang -o ned-vlan.html
```

Make particular note of the **path** in the right column, if one removes the namespace (e.g. `ios:`) it actually already looks like a URL path! So, in our case, let's have a look at `GigabitEthernet3` and retrieve its configuration so that we can alter the data and create a new VLAN interface.

<p class='my-notify-info'>Note that without the pruning the native element model is super large. It produces approx. 60MB worth of HTML table code which takes a few minutes to render :-)</p>

In [ ]:
%%HTML
<iframe width="100%" height="400px" src='/notebooks/tmp/ned-vlan.html'</iframe>

In [ ]:
interface = pp(session.get(url('running/ios:native/ios:interface/GigabitEthernet/3'), params=dict(deep=True)))

To create a new VLAN interface, we need to take the above information and add a bit of information. Look up the `encapsulation` node in the above model tree. Make a note of the path. We should set

    /native/interface/GigabitEthernet/3/encapsulation/dot1Q/vlan-id
    

with the VLAN value.

In [ ]:
from collections import OrderedDict

int2 = OrderedDict(interface['ned:GigabitEthernet'])

encap = {'dot1Q': {'vlan-id': 100}}
ipv4 = {'address': {'primary': {'address': '172.16.101.1', 'mask': '255.255.255.0'}}}

int2['encapsulation'] = encap
int2['ip'] = ipv4
int2['name'] = '3.100'
int2.move_to_end('name', last=False)

# delete unwanted stuff
del int2['mop']
del int2['negotiation']

interface['ned:GigabitEthernet']=int2
o=pp(interface)

# We could of course just say:
new_int = json.loads('''
{
  "ned:GigabitEthernet": {
    "name": "100.3",
    "ip": {
      "address": {
        "primary": {
          "mask": "255.255.255.0",
          "address": "172.16.101.1"
        }
      }
    },
    "encapsulation": {
      "dot1Q": {
        "vlan-id": 100
      }
    }
  }
}''')

In [ ]:
o=session.patch(url('running/ios:native/ios:interface/ios:GigabitEthernet'), json=interface)
o.status_code

In [ ]:
interface = pp(session.get(url('running/ios:native/ios:interface'), params=dict(deep=True)))